In [31]:
import os
import sys
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

sys.path.append('../')
from src.datagen import DataGenerator
from src.utils import get_ytrue_ypred_targets, save_confusion_matrix, save_summary_plots
from src.convert import get_audio_path

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, Dropout, \
                                    Activation, MaxPooling2D, Flatten, \
                                    BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [32]:
datagen = DataGenerator('../data/test', include=['Rock','Hip-Hop','Instrumental'], 
                        batch_size=1, dim=(128,640), 
                        n_channels=1, test=True)

In [33]:
model = load_model('../models/model4_arch_and_weights.h5')

In [34]:
y_pred_softmax = model.predict_generator(datagen)
y_pred = np.argmax(y_pred_softmax, axis=1)

In [35]:
y_pred.shape

(300,)

In [36]:
y_true = datagen.test_labels
y_true = np.argmax(y_true, axis=1)

In [37]:
y_true.shape

(300,)

In [38]:
correct = y_true == y_pred
mask = correct == False

In [39]:
mask.shape

(300,)

In [40]:
files = datagen.files

In [41]:
files.shape

(300,)

In [42]:
misclass = pd.DataFrame()
misclass['file'] = files[mask]
misclass['pred'] = y_pred[mask]
misclass['true'] = y_true[mask]

In [43]:
# misclass.file[0].split('/')[1][:6]

In [44]:
AUDIO_DIR = '../data/fma_small'

In [45]:
# get_audio_path(AUDIO_DIR, 116390)

In [46]:
misclass

,file,pred,true
0,Hip-Hop/116390.npy,2,0
1,Hip-Hop/147020.npy,2,0
2,Hip-Hop/111938.npy,2,0
3,Hip-Hop/148032.npy,2,0
4,Hip-Hop/042040.npy,2,0
5,Hip-Hop/112485.npy,2,0
6,Hip-Hop/116407.npy,1,0
7,Hip-Hop/110777.npy,2,0
8,Instrumental/134794.npy,2,1
9,Instrumental/134790.npy,2,1


In [54]:
files[mask][28]

'Instrumental/134796.npy'

In [53]:
y_pred_softmax[mask][28]

array([0.9361285 , 0.03499081, 0.02888075], dtype=float32)

In [14]:
tracks = pd.read_csv('../data/fma_metadata/tracks.csv', index_col=0, header=[0, 1]) 

In [15]:
keep_cols = [('set', 'split'),
('set', 'subset'),('track', 'genre_top')]

df_all = tracks[keep_cols]
df_all = df_all[df_all[('set', 'subset')] == 'small']
df_all['track_id'] = df_all.index

In [16]:
rock = (df_all[('track', 'genre_top')]=='Rock')
inst = (df_all[('track', 'genre_top')]=='Instrumental')
hiphop = (df_all[('track', 'genre_top')]=='Hip-Hop')
mask = rock | inst | hiphop
genres = df_all[mask]

In [17]:
genres[genres.track_id == 134796]

set                track track_id
         split subset     genre_top         
track_id                                    
134796    test  small  Instrumental   134796